In [1]:
from __future__ import unicode_literals, print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd

import os
import re
import random

In [31]:
#Normalize every sentence
def normalize_sentence(df, lang):
    sentence = df[lang].str
    sentence = sentence.encode('utf-8', errors='ignore').str.decode('utf-8')
    return sentence

def read_sentence(df, first, second):
   sentence1 = normalize_sentence(df, first)
   sentence2 = normalize_sentence(df, second)
   return sentence1, sentence2

def read_file(loc, first, second):
   df = pd.read_csv(loc, delimiter=' , ', header=None, names=[first, second])
   return df

def process_data(first, second):
    df = read_file('/content/ferdosi.txt', first, second)
    print("Read %s sentence pairs" % len(df))
    sentence1, sentence2 = read_sentence(df, first, second)

    source = Lang()
    target = Lang()
    pairs = []

    for i in range(len(df)):
        if len(sentence1[i].split(' ')) < MAX_LENGTH and len(sentence2[i].split(' ')) < MAX_LENGTH:
            full = [sentence1[i], sentence2[i]]
            source.addSentence(sentence1[i])
            target.addSentence(sentence2[i])
            pairs.append(full)

    return source, target, pairs

In [2]:
SOS_token = 0
EOS_token = 1
MAX_LENGTH = 15

#initialize Lang Class
class Lang:
    def __init__(self):
        #initialize containers to hold the words and corresponding index
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    #split a sentence into words and add it to the container
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    #If the word is not in the container, the word will be added to it, 
    #else, update the word counter
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [3]:
def indexesFromSentence(lang, sentence):
   return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
   indexes = indexesFromSentence(lang, sentence)
   indexes.append(EOS_token)
   return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(source, target, pair):
   input_tensor = tensorFromSentence(source, pair[0])
   target_tensor = tensorFromSentence(target, pair[1])
   return (input_tensor, target_tensor)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
class Encoder(nn.Module):
   def __init__(self, input_dim, hidden_dim, embbed_dim, num_layers):
       super(Encoder, self).__init__()
      
       #set the encoder input dimesion , embbed dimesion, hidden dimesion, and number of layers 
       self.input_dim = input_dim
       self.embbed_dim = embbed_dim
       self.hidden_dim = hidden_dim
       self.num_layers = num_layers

       #initialize the embedding layer with input and embbed dimention
       self.embedding = nn.Embedding(input_dim, self.embbed_dim)
       #intialize the GRU to take the input dimetion of embbed, and output dimention of hidden and
       #set the number of GRU layers
       self.GRU = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
              
   def forward(self, src):
      
       embedded = self.embedding(src).view(1,1,-1)
       outputs, hidden = self.GRU(embedded)
       return outputs, hidden

In [6]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, embbed_dim, num_layers):
        super(Decoder, self).__init__()

        #set the encoder output dimension, embed dimension, hidden dimension, and number of layers 
        self.embbed_dim = embbed_dim
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.dropout = torch.nn.Dropout(0.1)

        # initialize every layer with the appropriate dimension. For the decoder layer, it will consist of an embedding, GRU, a Linear layer and a Log softmax activation function.
        self.embedding = nn.Embedding(output_dim, self.embbed_dim)
        self.GRU = nn.GRU(self.embbed_dim, self.hidden_dim, num_layers=self.num_layers)
        self.out = nn.Linear(self.hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
      
    def forward(self, input, hidden):
        

        # reshape the input to (1, batch_size)
        input = input.view(1, -1)
        embedded = self.embedding(input)
        embedded = self.dropout(embedded)
        output, hidden = self.GRU(embedded, hidden)       
        prediction = self.softmax(self.out(output[0]))
      
        return prediction, hidden


In [7]:
class Seq2Seq(nn.Module):
   def __init__(self, encoder, decoder, device, MAX_LENGTH=MAX_LENGTH):
        super().__init__()
      
        #initialize the encoder and decoder
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
     
   def forward(self, source, target, teacher_forcing_ratio=0.5):

        input_length = source.size(0) #get the input length (number of words in sentence)
        target_length = target.shape[0]
        batch_size = target.shape[1] 
        vocab_size = self.decoder.output_dim
      
        #initialize a variable to hold the predicted outputs
        outputs = torch.zeros(target_length, batch_size, vocab_size).to(self.device)

        #encode every word in a sentence
        encoder_output, encoder_hidden = self.encoder(source)

        #use the encoder’s hidden layer as the decoder hidden
        decoder_hidden = encoder_hidden.to(device)
    
        #add a token before the first predicted word
        decoder_input = torch.tensor([SOS_token], device=device)  # SOS

        #topk is used to get the top K value over a list
        #predict the output word from the current target word. If we enable the teaching force,  then the #next decoder input is the next word, else, use the decoder output highest value. 

        for t in range(target_length):   
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
            outputs[t] = decoder_output
            teacher_force = random.random() < teacher_forcing_ratio
            topv, topi = decoder_output.topk(1)
            input = (target[t] if teacher_force else topi)
            if(teacher_force == False and input.item() == EOS_token):
                break

        return outputs

In [38]:
teacher_forcing_ratio = 0.5

def clacModel(model, input_tensor, target_tensor, model_optimizer, criterion):
    
    model_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    loss = 0
    epoch_loss = 0

    output = model(input_tensor, target_tensor)

    num_iter = output.size(0)

    #calculate the loss from a predicted sentence with the expected result
    for ot in range(num_iter):
        loss += criterion(output[ot], target_tensor[ot])

    loss.backward()
    model_optimizer.step()
    epoch_loss = loss.item() / num_iter

    return epoch_loss

def trainModel(model, source, target, pairs, num_iteration=20000):
    model.train()

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.NLLLoss()
    total_loss_iterations = 0
    loss = 0
    training_pairs = [tensorsFromPair(source, target, random.choice(pairs)) for i in range(num_iteration)]
    
    print("num of iterations is", num_iteration)
    for iter in range(1, num_iteration + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        total_loss_iterations += clacModel(model, input_tensor, target_tensor, optimizer, criterion)
        loss += total_loss_iterations
        
        if iter % 100 == 0:
            avarage_loss = total_loss_iterations / 50
            total_loss_iterations = 0
            print('%d %.4f' % (iter, avarage_loss))
            print(f'[Train] loss: {loss / (i + 1):.3e}')
        
            
    torch.save(model.state_dict(), 'mytraining.pt')
    return model , (loss / iter)

In [48]:
def evaluate(model, input_lang, output_lang, sentences, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentences[0])
        output_tensor = tensorFromSentence(output_lang, sentences[1])
    
        decoded_words = []
    
        output = model(input_tensor, output_tensor)
    
        for ot in range(output.size(0)):
            topv, topi = output[ot].topk(1)

            if topi[0].item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi[0].item()])
    return decoded_words

def evaluateRandomly(model, source, target, pairs, n=5):
    for i in range(n):
        pair = random.choice(pairs)
        print("source {}".format(pair[0]))        
        output_words = evaluate(model, source, target, pair)
        output_sentence = ' '.join(output_words)
        print("target {}".format(pair[1]))
        print("predicted {}".format(output_sentence))
        


In [49]:
source, target, pairs = process_data('first', 'second')

randomize = random.choice(pairs)
print('random sentence {}'.format(randomize))

#print number of words
input_size = source.n_words
output_size = target.n_words
print('Input : {} Output : {}'.format(input_size, output_size))

embed_size = 256
hidden_size = 512
num_layers = 1
num_iteration = 1000

#create encoder-decoder model
encoder = Encoder(input_size, hidden_size, embed_size, num_layers)
decoder = Decoder(output_size, hidden_size, embed_size, num_layers)

model = Seq2Seq(encoder, decoder, device).to(device)

#print model 
print(encoder)
print(decoder)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  del sys.path[0]


Read 49609 sentence pairs
random sentence ['که ما سربسر مر تو را بنده ایم', 'بفرمان و رایت سرافگنده ایم']
Input : 12458 Output : 13071
Encoder(
  (embedding): Embedding(12458, 256)
  (GRU): GRU(256, 512)
)
Decoder(
  (dropout): Dropout(p=0.1, inplace=False)
  (embedding): Embedding(13071, 256)
  (GRU): GRU(256, 512)
  (out): Linear(in_features=512, out_features=13071, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [50]:
from time import time

epochs = 3

for i in range(epochs):

    start_time = time()
    
    model, train_loss = trainModel(model, source, target, pairs, num_iteration)
    evaluateRandomly(model, source, target, pairs)


    print(f'\tTrain Loss: {train_loss:.3e} | Train PPL: {np.exp(train_loss):.3e}')

    end_time = time()

    print("ended in: ", end_time - start_time, "sec")

num of iterations is 1000
100 12.4936
[Train] loss: 3.308e+04
200 11.0764
[Train] loss: 6.035e+04
300 11.0905
[Train] loss: 8.793e+04
400 11.2645
[Train] loss: 1.165e+05
500 11.7843
[Train] loss: 1.466e+05
600 11.4418
[Train] loss: 1.759e+05
700 11.3824
[Train] loss: 2.047e+05
800 11.7607
[Train] loss: 2.342e+05
900 12.1450
[Train] loss: 2.653e+05
1000 11.7166
[Train] loss: 2.947e+05
source ز بیژن مگر اگهی یابما
['ز بیژن مگر اگهی یابما', 'بدین کار هشیار بشتابما']
target بدین کار هشیار بشتابما
predicted به به هر هر <EOS>
source هم اکنون تن کشتگان را بخاک
['هم اکنون تن کشتگان را بخاک', 'بپوشید جایی که باشد مغاک']
target بپوشید جایی که باشد مغاک
predicted به به هر <EOS>
source جهانجوی را دید جامی به دست
['جهانجوی را دید جامی به دست', 'نگهبان اسپان همه خفته مست']
target نگهبان اسپان همه خفته مست
predicted به به هر <EOS>
source مرا ناگهان در عماری نشاند
['مرا ناگهان در عماری نشاند', 'بران خوب چهره فسونی بخواند']
target بران خوب چهره فسونی بخواند
predicted به به هر <EOS>
source به تو داد یک 